In [8]:
from matplotlib import pyplot as plt
import pandas as pd
import altair as alt
from opentrend.dbmanage import DBManger

In [9]:

sql = """
SELECT `language` as lan,created_day,period,sum(period_stars_count) as stars
FROM lake.opentrend
where `hour`= 18
-- and created_day >= "2020-01-02"
and period = "monthly"
group by 1,2,3
"""


def getDataset():
    dbm = DBManger()
    data, fields = dbm.query(sql)
    df = pd.DataFrame(data, columns=fields)
    return df

In [10]:

df = getDataset()
mdf = df[df["period"] == "monthly"]
mdf["stars"] = mdf["stars"].astype("float64")
table = pd.pivot_table(mdf, "stars", "created_day", "lan")
table.fillna(0, inplace=True)
smoothTable = table.rolling(window=30).mean()
topLan = smoothTable.sum(0).sort_values(ascending=False)[:20].index.tolist()
smoothTable = smoothTable.loc[:, topLan]


In [11]:
table = smoothTable
perTable = (table.T / table.sum(1)).T

In [15]:
perTable.fillna(0,inplace=True)

In [18]:
perTable

lan,Python,Java,JavaScript,Jupyter Notebook,Go,Shell,TypeScript,Vue,GLSL,Dart,C++,C,V,PHP,C#,Swift,HTML,Rust,Clojure,Scala
created_day,,,,,,,,,,,,,,,,,,,,
2019-09-18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2019-09-19,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2019-09-20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2019-09-21,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2019-09-22,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-02,0.170076,0.037188,0.314543,0.018262,0.062962,0.022407,0.122822,0.0,0.0,0.0,0.151219,0.016570,0.0,0.000851,0.051913,0.007342,0.001281,0.022563,0.0,0.0
2020-03-03,0.167319,0.037186,0.316704,0.018185,0.064635,0.023654,0.115313,0.0,0.0,0.0,0.151733,0.017500,0.0,0.000422,0.053930,0.008947,0.001276,0.023196,0.0,0.0
2020-03-04,0.168103,0.038586,0.312062,0.018422,0.073403,0.024885,0.109728,0.0,0.0,0.0,0.144811,0.018467,0.0,0.000742,0.055891,0.010539,0.001270,0.023092,0.0,0.0


In [22]:
# perTable.plot()
c = alt.Chart(perTable.reset_index()).mark_line().encode(x="created_day",y="Python")
c.interactive()

alt.Chart(...)